   Copyright 2018 Google Inc.

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
   
   Author: Pramod Kaushik Mudrakarta (pramodkm@uchicago.edu)

# Analysis of Neural Programmer

## Part of this notebook use code from: https://github.com/pramodkaushik/acl18_results

In [0]:
import copy
import itertools
import operator
import os
import pickle
import re
import string
import sys
import time
from collections import Counter, defaultdict
from random import shuffle
import spacy
import autoreload
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from IPython.display import HTML, Image, clear_output, display
from scipy.spatial.distance import cosine

sys.path.append('../neural_programmer')

/scratch/pramodkm/tensorflow_gpu_python3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
import notebook_utils
import data_utils
from neural_programmer import evaluate

In [0]:
%reload_ext autoreload
%autoreload 2

## Paths, parameters, etc. 

In [0]:
# Use only one GPU on the multi-GPU machine
# os.environ["CUDA_VISIBLE_DEVICES"] = "1,3"

# WikiTableQuestions data
DATA_DIR = '../wtq_data'
PERTURBED_DATA_DIR = '../perturbed_wtq_data'

# Pretrained model
MODEL_FILE = os.path.join('..', 'pretrained_model', 'model_92500')
model_step = int(MODEL_FILE.split('_')[-1])

# Output directory to write attributions
OUT_DIR = '../results'

# Overstability curve file
OVERSTABILITY_CURVE_FILE = os.path.join(OUT_DIR, 'overstability.eps')

pd.options.display.max_colwidth=100000

### Some utility functions

In [0]:
# Operators whose results builds upon the result of previously applied operators
acts_on_prev_result = {
    'count': True,
    'prev': True,
    'next': True,
    'first': True,
    'last': True,
    'mfe': False,
    'greater': False,
    'lesser': False,
    'geq': False,
    'leq': False,
    'max': True,
    'min': True,
    'select': False,
    'reset': False,
    'print': True
}

# Operators whose result depends on the column it is acting on
relies_on_col = {
    'count': False,
    'prev': False,
    'next': False,
    'first': False,
    'last': False,
    'mfe': True,
    'greater': True,
    'lesser': True,
    'geq': True,
    'leq': True,
    'max': True,
    'min': True,
    'select': True,
    'reset': False,
    'print': True
    
}

def get_program_mask(program, ignore_answer_cond=False):
    """ 
    Returns a mask indicating attributions to which ops/cols are considered significant.
    The conditions for are:
    1) affect answer computation, (toggled by "ignore_answer_cond")
    2) are not the same as their table-specific default counterparts
    
    program = [op (default_op), col (default_col)] * 4 
    """
    mask = [False] * (2 * 4)
    for i in range(3, -1, -1):
        op, default_op = program[2*i].split('(')
        op = op.strip()
        default_op = default_op.strip().strip(')')
        mask[2*i] = (op != default_op)
        col, default_col = program[2*i + 1].split('(')
        col = col.strip()
        default_col = default_col.strip().strip(')')
        if ignore_answer_cond:
            continue
        if relies_on_col[op]:
            mask[2*i+1] = (col != default_col)
        if not acts_on_prev_result[op]:
            break
    return mask

## Load data, build graph and restore pretrained weights

In [0]:
train_data, dev_data, test_data, utility, unprocessed_dev_data = notebook_utils.init_data(DATA_DIR)
num_dev_examples = 2831

('Annotated examples loaded ', 14152)
('Annotated examples loaded ', 4344)
('entry match token: ', 9133, 9133)
('entry match token: ', 9134, 9134)
('# train examples ', 10178)
('# dev examples ', 2546)
('# test examples ', 3913)


In [0]:
tf.reset_default_graph()
sess, graph, params = notebook_utils.build_graph(utility)

forget gate bias
('step: ', 0)
('step: ', 1)
('step: ', 2)
('step: ', 3)
('optimize params ', ['unit', 'word', 'word_match_feature_column_name', 'controller', 'column_controller', 'column_controller_prev', 'controller_prev', 'question_lstm_ix', 'question_lstm_fx', 'question_lstm_cx', 'question_lstm_ox', 'question_lstm_im', 'question_lstm_fm', 'question_lstm_cm', 'question_lstm_om', 'question_lstm_i', 'question_lstm_f', 'question_lstm_c', 'question_lstm_o', 'history_recurrent', 'history_recurrent_bias', 'break_conditional'])
('grads: ', <tf.Tensor 'gradients_40/L2Loss_grad/mul:0' shape=(15, 256) dtype=float64>, 'unit')
('grads: ', <tf.Tensor 'gradients_40/L2Loss_1_grad/mul:0' shape=(10800, 256) dtype=float64>, 'word')
('grads: ', <tf.Tensor 'gradients_40/L2Loss_2_grad/mul:0' shape=(1,) dtype=float64>, 'word_match_feature_column_name')
('grads: ', <tf.Tensor 'gradients_40/L2Loss_3_grad/mul:0' shape=(512, 256) dtype=float64>, 'controller')
('grads: ', <tf.Tensor 'gradients_40/L2Loss_4_gra

In [0]:
sess, graph = notebook_utils.restore_model(sess, graph, params, MODEL_FILE)

INFO:tensorflow:Restoring parameters from ../pretrained_model/model_92500


In [0]:
num_correct, num_examples, correct_dict = evaluate(sess, dev_data, utility.FLAGS.batch_size, graph, model_step)

('dev set accuracy   after ', 92500, ' : ', 0.37195600942655144)
(2546, 2546)
--------


In [0]:
print("Validation accuracy:", num_correct/float(num_dev_examples))

Validation accuracy: 0.3345107735782409


## Apply Integrated Gradients (IG) 
Note: attributions are available already in "results". Hence, to reproduce the results of the ACL paper, one can skip this part.

In [0]:
# write attributions to this folder
attrs_outdir = os.path.join(OUT_DIR, 'attributions')
if not os.path.isdir(attrs_outdir):
    os.makedirs(attrs_outdir)

# get embedding of dummy token
embeddings = graph.params["word"].eval()
dummy_embedding = embeddings[utility.dummy_token_id, :]

# which data to use?
data = dev_data

# number of sample points for Riemann integral computation
num_points = 2000

# hard coded stuff in the code
question_attention_mask_value = -10000.0

In [0]:
batch_size = graph.batch_size

for offset in range(0, len(data) - graph.batch_size + 1, graph.batch_size):
    feed_dict = data_utils.generate_feed_dict(data, offset, graph.batch_size, graph)

    # first run inference to get operator and column sequences, and embeddings of question words
    fetches = [graph.final_correct_list, graph.final_operation_softmax,
               graph.final_column_softmax, graph.question_words_embeddings]
    correct_list, operation_softmax, column_softmax, question_words_embeddings = sess.run(
        fetches, feed_dict)

    # compute table-specific default programs for tables in this batch
    feed_copy = feed_dict.copy()
    for t in graph.question_words_embeddings:
        feed_copy[t] = np.concatenate(
            [np.expand_dims(dummy_embedding, 0)]*batch_size, 0)

    # Ideally the following line should be uncommented, but for attributions,
    # we choose to keep this variable fixed. Note that this induces some bias
    # in the attributions as the baseline is no longer an "empty" question, but
    # an empty question where the question length is implicitly encoded in this variable
    # feed_copy[graph.batch_question_attention_mask].fill(question_attention_mask_value)

    feed_copy[graph.batch_exact_match] = np.zeros_like(
        feed_copy[graph.batch_exact_match])
    feed_copy[graph.batch_column_exact_match] = np.zeros_like(
        feed_copy[graph.batch_column_exact_match])

    fetches = [graph.final_operation_softmax, graph.final_column_softmax]

    default_operation_softmax, default_column_softmax = sess.run(
        fetches, feed_copy)

    for batch_id in range(batch_size):
        wiki_example = data[offset+batch_id]

        # get operator indices
        op_indices = np.argmax(operation_softmax[batch_id, :, :], axis=1)
        col_indices = np.argmax(column_softmax[batch_id, :, :], axis=1)

        op_list = notebook_utils.softmax_to_names(
            operation_softmax[batch_id, :, :], utility.operations_set)
        col_list = notebook_utils.softmax_to_names(
            column_softmax[batch_id, :, :], notebook_utils.get_column_names(wiki_example))

        default_op_list = notebook_utils.softmax_to_names(
            default_operation_softmax[batch_id, :, :], utility.operations_set)
        default_col_list = notebook_utils.softmax_to_names(
            default_column_softmax[batch_id, :, :], notebook_utils.get_column_names(wiki_example))

        print([notebook_utils.rename(w) for w in op_list])
        print(col_list)

        # Sample points along the integral path and collect them as one batch
        scaled_feed = feed_dict.copy()
        for key in list(scaled_feed.keys()):
            value = feed_dict[key]
            if key.shape[0] == batch_size:  # this is a hack
                scaled_feed[key] = [value[batch_id] for i in range(batch_size)]
        scaled_feed[graph.op_ids] = op_indices
        scaled_feed[graph.col_ids] = col_indices

        num_examples = batch_size * int(num_points/float(batch_size))
        scale = 1.0/num_examples

        batch_op_attribution = np.zeros(
            [graph.max_passes, graph.question_length+2], dtype=np.float32)
        batch_col_attribution = np.zeros(
            [graph.max_passes, graph.question_length+2], dtype=np.float32)

        attr_op_softmax = []
        attr_col_softmax = []

        actual_num_numeric_cols = len(wiki_example.original_nc_names)
        actual_num_word_cols = len(wiki_example.original_wc_names)

        exact_match = wiki_example.exact_match
        exact_column_match = wiki_example.exact_column_match

        batch_question_embeddings = np.array(question_words_embeddings)[
            :, batch_id, :]  # shape: 62 x 256

        # split up set of points into batch_size'd batches
        for k in range(0, num_examples, batch_size):
            print('k:', k)
            # scale question words to points between dummy_embedding and actual embedding
            qw_jump = [None]*graph.question_length
            for i, t in enumerate(graph.question_words_embeddings):
                qw_jump[i] = scale * \
                    (batch_question_embeddings[i] - dummy_embedding)
                scaled_feed[t] = [dummy_embedding + j*qw_jump[i]
                                  for j in range(k, k+batch_size)]

            # scale batch_exact_match
            scaled_exact_match = []
            scaled_column_exact_match = []

            exact_match_jump = [None]*(graph.num_cols + graph.num_word_cols)
            exact_column_match_jump = [None] * \
                (graph.num_cols + graph.num_word_cols)
            for i in range(graph.num_cols):
                if i < actual_num_numeric_cols:  # do not scale dummy columns
                    scaled_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_match[i]) for j in range(k, k+batch_size)], 1))
                    exact_match_jump[i] = scale*np.array(exact_match[i])
                    scaled_column_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_column_match[i]) for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[i] = scale * \
                        np.array(exact_column_match[i])
                else:
                    scaled_exact_match.append(np.expand_dims(
                        [exact_match[i] for j in range(k, k+batch_size)], 1))
                    exact_match_jump[i] = 0
                    scaled_column_exact_match.append(np.expand_dims(
                        [exact_column_match[i] for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[i] = 0

            for i in range(graph.num_word_cols):
                if i < actual_num_word_cols:  # do not scale dummy column names
                    scaled_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_match[graph.num_cols+i]) for j in range(k, k+batch_size)], 1))
                    exact_match_jump[graph.num_cols + i] = scale * \
                        np.array(exact_match[graph.num_cols+i])
                    scaled_column_exact_match.append(np.expand_dims(
                        [j*scale*np.array(exact_column_match[graph.num_cols + i]) for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[graph.num_cols + i] = scale * \
                        np.array(exact_column_match[graph.num_cols + i])
                else:
                    scaled_exact_match.append(np.expand_dims(
                        [exact_match[graph.num_cols+i] for j in range(k, k+batch_size)], 1))
                    exact_match_jump[graph.num_cols + i] = 0
                    scaled_column_exact_match.append(np.expand_dims(
                        [exact_column_match[graph.num_cols + i] for j in range(k, k+batch_size)], 1))
                    exact_column_match_jump[graph.num_cols + i] = 0

            scaled_feed[graph.batch_exact_match] = np.concatenate(
                scaled_exact_match, 1)  # shape 20 x 40 x 100
            scaled_feed[graph.batch_column_exact_match] = np.concatenate(
                scaled_column_exact_match, 1)  # shape 20 x 40

            # compute gradients
            fetches = [graph.final_operation_softmax, graph.final_column_softmax, graph.operator_gradients,
                       graph.column_gradients]
            temp_op_softmax, temp_col_softmax, operator_gradients, column_gradients = sess.run(
                fetches, scaled_feed)  # operator gradient shape: 4 x 62 x 20 x 256

            attr_op_softmax.append(temp_op_softmax)
            attr_col_softmax.append(temp_col_softmax)

            # compute attributions
            for stage in range(graph.max_passes):
                n = int(len(operator_gradients)/graph.max_passes)
                temp = [np.sum(operator_gradients[n*stage][i]*qw_jump[i], axis=(0, 1))
                        for i in range(graph.question_length)]
                temp += [np.sum([operator_gradients[n*stage+1][0][:, i, :]*exact_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                temp += [np.sum([operator_gradients[n*stage+2][0][:, i]*exact_column_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                batch_op_attribution[stage, :] += temp

            for stage in range(graph.max_passes):
                n = int(len(column_gradients)/graph.max_passes)
                temp = [np.sum(column_gradients[n*stage][i]*qw_jump[i], axis=(0, 1))
                        for i in range(graph.question_length)]
                temp += [np.sum([column_gradients[n*stage+1][0][:, i, :]*exact_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                temp += [np.sum([column_gradients[n*stage+2][0][:, i]*exact_column_match_jump[i]
                                 for i in range(graph.num_cols + graph.num_word_cols)])]
                batch_col_attribution[stage, :] += temp

        # sanity check to make sure the integral summation adds up to function difference
        attr_op_softmax = np.concatenate(attr_op_softmax, axis=0)
        attr_col_softmax = np.concatenate(attr_col_softmax, axis=0)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_op_attribution[stage, :])
            input_fn_value = operation_softmax[batch_id,
                                               stage, op_indices[stage]]
            baseline_fn_value = attr_op_softmax[0, stage, op_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('OP', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=',
                  input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs-rhs)
        for stage in range(graph.max_passes):
            lhs = np.sum(batch_col_attribution[stage, :])
            input_fn_value = column_softmax[batch_id,
                                            stage, col_indices[stage]]
            baseline_fn_value = attr_col_softmax[0, stage, col_indices[stage]]
            rhs = input_fn_value - baseline_fn_value
            print('COL', stage, ':', 'baseline=', baseline_fn_value, ', input_fn=',
                  input_fn_value, 'check: ', lhs, ' - ', rhs, ' = ', lhs-rhs)

        op_attributions = [None]*graph.max_passes
        question_begin = np.nonzero(
            wiki_example.question_attention_mask)[0].shape[0]

        attributions_matrix = np.zeros(
            [graph.question_length - question_begin + 2, 2 * graph.max_passes])
        row_labels = []  # question words, tm, cm
        col_labels = []  # operator and column selections
        col_label_softmaxes = []  # softmaxes of the selections

        for ix in range(question_begin, graph.question_length):
            word = utility.reverse_word_ids[wiki_example.question[ix]]
            if word == utility.unk_token:
                word = word + '-' + [str(w) for w in wiki_example.string_question if w !=
                                     wiki_example.question_number and w != wiki_example.question_number_1][ix - question_begin]
            word = notebook_utils.rename(word)
            row_labels.append(word)
        row_labels.extend(['tm', 'cm'])

        for stage in range(graph.max_passes):
            col_labels.append(notebook_utils.rename(
                op_list[stage]) + ' (' + notebook_utils.rename(default_op_list[stage]) + ')')
            col_labels.append(notebook_utils.rename(
                col_list[stage]) + ' (' + notebook_utils.rename(default_col_list[stage]) + ')')

            col_label_softmaxes.append(str(operation_softmax[batch_id, stage, op_indices[stage]]) + ' (' + str(
                default_operation_softmax[batch_id, stage, op_indices[stage]]) + ')')
            col_label_softmaxes.append(str(column_softmax[batch_id, stage, col_indices[stage]]) + ' (' + str(
                default_column_softmax[batch_id, stage, col_indices[stage]]) + ')')

            attributions_matrix[:, 2 * stage] = batch_op_attribution[stage, question_begin:]
            attributions_matrix[:, 2 * stage +
                                1] = batch_col_attribution[stage, question_begin:]

        question_string = ' '.join([notebook_utils.rename(str(w))
                                    for w in wiki_example.string_question])

        # save operator and column selections to file
        with tf.gfile.GFile(os.path.join(attrs_outdir, wiki_example.question_id + '_labels.tsv'), 'w') as outf:
            outf.write(question_string)
            outf.write('\n')
            outf.write(str(correct_list[batch_id] == 1.0))
            outf.write('\n')
            outf.write('\t'.join(row_labels) + '\n')
            outf.write('\t'.join(col_labels) + '\n')
            outf.write('\t'.join(col_label_softmaxes) + '\n')

        # save attributions to file
        np.savetxt(os.path.join(
            attrs_outdir, wiki_example.question_id + '_attrs.txt'), attributions_matrix)


In [0]:
data = dev_data
attrs_outdir = os.path.join(OUT_DIR, 'attributions')
figs_outdir = os.path.join(OUT_DIR, "heatmaps")
if not os.path.isdir(figs_outdir):
    os.makedirs(figs_outdir)

In [0]:
sns.set(font_scale=1.0)
rc={'axes.labelsize': 11, 'xtick.labelsize': 14, 'ytick.labelsize': 14}
sns.set(rc=rc)
for wiki_example in data[:graph.batch_size*int(len(data)/graph.batch_size)]:
    attributions = np.loadtxt(os.path.join(attrs_outdir, wiki_example.question_id + '_attrs.txt'))
    with tf.gfile.GFile(os.path.join(attrs_outdir, wiki_example.question_id + '_labels.tsv')) as f:
        lines = f.readlines()
        xlabels = ['\n'.join(w.split()) for w in lines[3].strip().split('\t')]
        ylabels = lines[2].strip().split('\t')
    mask = get_program_mask(lines[3].strip().split('\t'))
    mask = np.expand_dims(mask, 0)
    plt.figure(figsize=(len(xlabels),len(ylabels)/2))
    plot_data = attributions/attributions.sum(axis=0)*mask
    with sns.axes_style('white'):
        sns.heatmap(plot_data, cbar=False, xticklabels=xlabels, yticklabels=ylabels, annot=True, fmt='.2f', robust=True)
    plt.tight_layout()
    plt.savefig(os.path.join(figs_outdir, wiki_example.question_id + '.png'))
    plt.close()

In [0]:
with tf.gfile.GFile(os.path.join(OUT_DIR, 'visualizations.html'), 'w') as htmlf:
    html_str = '<head><link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" integrity="sha384-Gn5384xqQ1aoWXA+058RXPxPg6fy4IWvTNh0E263XmFcJlSAwiGgFAW/dAiS6JXm" crossorigin="anonymous"></head>'
    html_str += '<body> <div class="container"> <h3> Visualizations of the attributions for the Neural Programmer network <br> <small> Lighter colors indicate high values <br> Green and red questions indicate whether the network got the answer right (or wrong)</small></h3></div><br>'
    html_str += '<div class="container">'
    for wiki_example in data[:graph.batch_size*int(len(data)/graph.batch_size)]:
        with tf.gfile.GFile(os.path.join(attrs_outdir, wiki_example.question_id + '_labels.tsv')) as f:
                lines = f.readlines()
                html_str += wiki_example.question_id + ' <div class=' + ('"text-success"' if lines[1].strip() == 'True' else '"text-danger"') + '>' + lines[0] + '</div><br>'
                html_str += '<img src="heatmaps/' + wiki_example.question_id + '.png"></img><br><hr><br>'
    
    html_str += '</div></body></html>'
    htmlf.write(html_str)
    print("Visualizations written to",os.path.join(OUT_DIR, 'visualizations.html'))

Visualizations written to ../results/visualizations.html
